In [0]:
pip install pyspark

Python interpreter will be restarted.
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488511 sha256=bcb2d8e9cdadd1daee9e68e723a96a7221b4559cc55cb4f2b4fbc46dd17f0595
  Stored in directory: /root/.cache/pip/wheels/92/09/11/aa01d01a7f005fda8a66ad71d2be7f8aa341bddafb27eee3c7
Successfully built pyspark
Python interpreter will be restarted.


In [0]:
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName('logistic-regression').getOrCreate()

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df=spark.read.csv( "/FileStore/tables/Churn_Modelling-2.csv",header=True,inferSchema=True)

In [0]:
df.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93

In [0]:
df=df.drop('CustomerId','Surname')

In [0]:
df.show()

+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|     0|
|        5|        850|    Spain|Female| 43|     2|125510.82|            1|        1|             1|        79084.1|     0|
|       

In [0]:
df=df.drop('RowNumber')

In [0]:
df.show()

+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|     0|
|        850|    Spain|Female| 43|     2|125510.82|            1|        1|             1|        79084.1|     0|
|        645|    Spain|  Male| 44|     8|113755.78|            2|        1|             

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
string=StringIndexer(inputCols=['Geography','Gender'],
outputCols=['geography','gender']
)

In [0]:
df=string.fit(df).transform(df)

In [0]:
df.show()

+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|CreditScore|geography|gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        619|      0.0|   1.0| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        608|      2.0|   1.0| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        502|      0.0|   1.0| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        699|      0.0|   1.0| 39|     1|      0.0|            2|        0|             0|       93826.63|     0|
|        850|      2.0|   1.0| 43|     2|125510.82|            1|        1|             1|        79084.1|     0|
|        645|      2.0|   0.0| 44|     8|113755.78|            2|        1|             

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
vector=VectorAssembler(inputCols=['CreditScore','geography','gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary'],
outputCol='independant_features'


)

In [0]:
df=vector.transform(df)

In [0]:
df.show()

+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+--------------------+
|CreditScore|geography|gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|independant_features|
+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+--------------------+
|        619|      0.0|   1.0| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|[619.0,0.0,1.0,42...|
|        608|      2.0|   1.0| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|[608.0,2.0,1.0,41...|
|        502|      0.0|   1.0| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|[502.0,0.0,1.0,42...|
|        699|      0.0|   1.0| 39|     1|      0.0|            2|        0|             0|       93826.63|     0|[699.0,0.0,1.0,39...|
|        850|      2.0|   1.0| 43|     2|125510.82|    

In [0]:
final_df=df.select('Exited','independant_features')

In [0]:
final_df.show()

+------+--------------------+
|Exited|independant_features|
+------+--------------------+
|     1|[619.0,0.0,1.0,42...|
|     0|[608.0,2.0,1.0,41...|
|     1|[502.0,0.0,1.0,42...|
|     0|[699.0,0.0,1.0,39...|
|     0|[850.0,2.0,1.0,43...|
|     1|[645.0,2.0,0.0,44...|
|     0|[822.0,0.0,0.0,50...|
|     1|[376.0,1.0,1.0,29...|
|     0|[501.0,0.0,0.0,44...|
|     0|[684.0,0.0,0.0,27...|
|     0|[528.0,0.0,0.0,31...|
|     0|[497.0,2.0,0.0,24...|
|     0|[476.0,0.0,1.0,34...|
|     0|[549.0,0.0,1.0,25...|
|     0|[635.0,2.0,1.0,35...|
|     0|[616.0,1.0,0.0,45...|
|     1|[653.0,1.0,0.0,58...|
|     0|[549.0,2.0,1.0,24...|
|     0|[587.0,2.0,0.0,45...|
|     0|[726.0,0.0,1.0,24...|
+------+--------------------+
only showing top 20 rows



In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
train_data,test_data=final_df.randomSplit([0.75,0.25])

In [0]:
train_data.show()

+------+--------------------+
|Exited|independant_features|
+------+--------------------+
|     0|(10,[0,1,3,6,9],[...|
|     0|(10,[0,2,3,6,9],[...|
|     0|(10,[0,2,3,6,9],[...|
|     0|(10,[0,2,3,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
+------+--------------------+
only showing top 20 rows



In [0]:
test_data.show()

+------+--------------------+
|Exited|independant_features|
+------+--------------------+
|     0|(10,[0,1,3,6,9],[...|
|     0|(10,[0,1,3,6,9],[...|
|     0|(10,[0,2,3,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
|     0|(10,[0,3,4,6,9],[...|
+------+--------------------+
only showing top 20 rows



In [0]:
classifier=LogisticRegression(featuresCol='independant_features',
                              labelCol='Exited'

)

In [0]:
classifier=classifier.fit(train_data)

In [0]:
classifier.coefficients

Out[39]: DenseVector([-0.0005, 0.0726, 0.5863, 0.0691, -0.022, 0.0, -0.0467, -0.0787, -1.0161, 0.0])

In [0]:
classifier.intercept

Out[40]: -3.9417546066237583

In [0]:
prediction=classifier.evaluate(test_data)

In [0]:
prediction.predictions.show()

+------+--------------------+--------------------+--------------------+----------+
|Exited|independant_features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(10,[0,1,3,6,9],[...|[1.75537260705270...|[0.85262916417010...|       0.0|
|     0|(10,[0,1,3,6,9],[...|[1.71835269158594...|[0.84791653091568...|       0.0|
|     0|(10,[0,2,3,6,9],[...|[0.38730143294085...|[0.59563290577674...|       0.0|
|     0|(10,[0,3,4,6,9],[...|[2.32487691912882...|[0.91091648504705...|       0.0|
|     0|(10,[0,3,4,6,9],[...|[2.88260492535870...|[0.94697980682505...|       0.0|
|     0|(10,[0,3,4,6,9],[...|[1.23474927547923...|[0.77464872743157...|       0.0|
|     0|(10,[0,3,4,6,9],[...|[2.19904822740750...|[0.90016400882308...|       0.0|
|     0|(10,[0,3,4,6,9],[...|[2.06528056755002...|[0.88748255290511...|       0.0|
|     0|(10,[0,3,4,6,9],[...|[1.95997460076782...|[0.87653020362901...|       0.0|
|   

In [0]:
prediction.accuracy

Out[43]: 0.8259075907590759